In [ ]:
import numpy as np
import pandas as pd
import openml

from tqdm.notebook import tqdm
from os import path

import sklearn
from sklearn import metrics
from sklearn import *


from func_timeout import func_timeout, FunctionTimedOut


import matplotlib.pyplot as plt
import random
import itertools as it
from scipy.sparse import lil_matrix

In [ ]:
def getDataset(openmlid):
    ds = openml.datasets.get_dataset(openmlid)
    df = ds.get_data()[0].dropna()
    y = df[ds.default_target_attribute].values
    
    categorical_attributes = df.select_dtypes(exclude=['number']).columns
    expansion_size = 1
    for att in categorical_attributes:
        expansion_size *= len(pd.unique(df[att]))
        if expansion_size > 10**5:
            break
    
    if expansion_size < 10**5:
        X = pd.get_dummies(df[[c for c in df.columns if c != ds.default_target_attribute]]).values.astype(float)
    else:
        print("creating SPARSE data")
        dfSparse = pd.get_dummies(df[[c for c in df.columns if c != ds.default_target_attribute]], sparse=True)
        
        print("dummies created, now creating sparse matrix")
        X = lil_matrix(dfSparse.shape, dtype=np.float32)
        for i, col in enumerate(dfSparse.columns):
            ix = dfSparse[col] != 0
            X[np.where(ix), i] = 1
        print("Done. shape is" + str(X.shape))
    return X, y

In [ ]:
def cross_validate(learner, X, y, train_size, repeats):
    scores = []
    n = X.shape[0]
    num_examples = int(train_size * n)
    
    for r in range(repeats):
        indices_train = random.sample(range(n), num_examples)
        learner.fit(X[indices_train,:], y[indices_train])
        indices_test = [i for i in range(n) if not i in indices_train]
        
        # compute validation error
        y_hat = learner.predict(X[indices_test])
        mistakes = 0
        for i, pred in enumerate(y_hat):
            act = y[indices_test[i]]
            if pred != act:
                mistakes += 1
        scores.append(mistakes / len(indices_test))
        
    return np.mean(scores)

def plot_learning_curves(learners, X, y, ax = None, train_portions = [0.05, 0.1, 0.2, 0.5, 0.7], repeats=10):
    total = len(learners) * len(train_portions)
    pbar = tqdm(total=total)
    for j, learner in enumerate(learners):
        scores_val = []
        slopes = []
        for i, train_portion in enumerate(train_portions):
            e_out = cross_validate(learner, X, y, train_portion, repeats)
            scores_val.append(e_out)
            if i > 0:
                slope = (e_out - scores_val[-2]) / (train_portion - train_portions[i-1])
                slopes.append(slope)
        
        if ax is None:
            fig, ax = plt.subplots()
        x_axis = np.round(X.shape[0] * np.array(train_portions)).astype(int)
        ax.plot(x_axis, scores_val, label=learner.__class__, color="C" + str(j))
        for i, train_portion in enumerate(train_portions):
            if i > 1 and slopes[i-1] < max(slopes[:i-1]):
                x = np.round(X.shape[0] * np.array([train_portions[i-1], train_portion])).astype(int)
                ax.fill_between(x, [0, 0], [scores_val[i-1], scores_val[i]], color="C" + str(j), alpha=0.2)
                
            pbar.update(1)
        ax.set_ylim([0,1])
    pbar.close()
    ax.legend()
    return ax


def gather_learning_curve_results(learners, datasets, train_portions, seeds):
    
    # read in data frame if exists
    FILENAME = "data/learningcurves.csv"
    cols = ["openmlid", "learner", "train_size", "seed", "error_rate"]
    df = pd.read_csv(FILENAME) if path.exists(FILENAME) else pd.DataFrame([], columns=cols)
    
    total = len(datasets) * len(learners) * len(train_portions) * len(seeds)
    pbar = tqdm(total=total)
    rows = []
    unsaved_changes = 0
    interrupted = False
    for openmlid in datasets:
        dsIndex = df["openmlid"] == openmlid
        
        print("DATASET", openmlid)
        X, y = getDataset(openmlid)
        n = X.shape[0]
        if n == 0:
            print("Omit empty dataset!")
        
        if interrupted:
            break
        
        for seed in seeds:
            if interrupted:
                break
            dsSeed = dsIndex & (df["seed"] == seed)
            for j, learner in enumerate(learners):
                if interrupted:
                    break
                dsLearner = dsSeed & (df["learner"] == str(learner))
                scores_val = []
                slopes = []
                for i, train_portion in enumerate(train_portions):
                    if interrupted:
                        break
                    random.seed(seed)
                    num_examples = train_portion if type(train_portion) == int else int(train_portion * n)
                    if n - num_examples < 100:
                        print("Dataset has only " + str(n) +" instances in total. That is not enough samples to train on " + str(num_examples) + ". Skipping")
                    else:
                        dsPortion = dsLearner & (df["train_size"] == num_examples)
                        if np.count_nonzero(dsPortion) == 0:
                            try:
                                indices_train = random.sample(range(n), num_examples)
                                indices_test = [i for i in range(n) if not i in indices_train]
                                indices_test = indices_test[:10000] # maximum 10k validation instances
                                X_train = X[indices_train]
                                y_train = y[indices_train]
                                X_test = X[indices_test]
                                y_test = y[indices_test]

                                inst = learner()
                                print("Training " + str(learner) + " on data of shape " + str(X_train.shape))
                                inst.fit(X_train, y_train)
                                print("Training ready. Obtaining predictions for " + str(X_test.shape[0]) + " instances.")
                                y_hat = inst.predict(X_test)
                                error_rate = 1 - sklearn.metrics.accuracy_score(y_test, y_hat)
                                row = [openmlid, str(learner), num_examples, seed, error_rate]
                                df.loc[len(df)] = row
                                unsaved_changes += 1

                                print(unsaved_changes, "unsaved changes")
                                if unsaved_changes >= 100:
                                    df.to_csv(FILENAME, index=False)
                                    unsaved_changes = 0
                            except KeyboardInterrupt:
                                print("Interrupted")
                                interrupted = True
                                break
                            except:
                                print("An error occurred on " + str(openmlid) + " with learner " + str(learner) + " under seed " + str(seed) + " for " + str(num_examples) + " examples.")
                    pbar.update(1)
    pbar.close()
    if unsaved_changes > 0:
        df.to_csv(FILENAME, index=False)
        unsaved_changes = 0
    return df
    

In [ ]:
datasets_dense = [1485, 1515, 1475, 1468, 1489, 23512, 23517, 40981, 40982, 40983, 40984, 40701, 40685, 40900,  1111, 40498, 41161, 41162, 41163, 41164, 41165, 41166, 41167, 41168, 41169, 41142, 41143, 41144, 41145, 41146, 41150, 41156, 41157, 41158,  41159, 41138, 54, 181, 188, 1461, 1494, 1464, 12, 23, 3, 1487, 40668, 1067, 1049, 40975, 31]
#1457
datasets_sparse = [1590, 1486, 4534, 4541, 4538, 4134, 4135, 40978, 40996, 41027, 40670, 42732, 42733, 42734, 41147]
datasets = datasets_dense

In [ ]:
learners = [
    (sklearn.svm.LinearSVC, {}),
    (sklearn.tree.DecisionTreeClassifier, {}),
    (sklearn.tree.ExtraTreeClassifier, {}),
    (sklearn.linear_model.LogisticRegression, {}),
    (sklearn.linear_model.PassiveAggressiveClassifier, {}),
    (sklearn.linear_model.Perceptron, {}),
    (sklearn.linear_model.RidgeClassifier, {}),
    (sklearn.linear_model.SGDClassifier, {}),
    (sklearn.neural_network.MLPClassifier, {}),
    (sklearn.discriminant_analysis.LinearDiscriminantAnalysis, {}),
    (sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis, {}),
    (sklearn.naive_bayes.BernoulliNB, {}),
    (sklearn.naive_bayes.MultinomialNB, {}),
    (sklearn.neighbors.KNeighborsClassifier, {}),
    (sklearn.ensemble.ExtraTreesClassifier, {}),
    (sklearn.ensemble.RandomForestClassifier, {}),
    (sklearn.ensemble.GradientBoostingClassifier, {})
]

In [ ]:
def get_bootstrap_samples(observations, n, stats=lambda x: np.mean(x)):
    if len(observations) <= 2:
        raise Exception("Cannot compute bootstrap sample of less than 2 observations!")
    bootstraps = []
    observations_as_list = list(observations)
    bootstrap_size = int(0.5 * len(observations_as_list))
    for i in range(n):
        sub_sample = random.sample(observations_as_list, bootstrap_size)
        bootstraps.append(stats(sub_sample))
    return bootstraps

In [ ]:
import scipy.stats
import time

def evaluate(learner, X, y, num_examples, seed=0):
    random.seed(seed)
    n = X.shape[0]
    indices_train = random.sample(range(n), num_examples)
    mask_train = np.zeros(n)
    mask_train[indices_train] = 1
    mask_train = mask_train.astype(bool)
    mask_test = (1 - mask_train).astype(bool)
    X_train = X[mask_train]
    y_train = y[mask_train]
    X_test = X[mask_test][:10000]
    y_test = y[mask_test][:10000]

    inst = learner()
    #print("Training " + str(learner) + " on data of shape " + str(X_train.shape))
    inst.fit(X_train, y_train)
    #print("Training ready. Obtaining predictions for " + str(X_test.shape[0]) + " instances.")
    y_hat = inst.predict(X_test)
    return 1 - sklearn.metrics.accuracy_score(y_test, y_hat)

def getSlopes(anchor_points, observations):
    slopes = []
    for i, o in enumerate(observations):
        if i > 0:
            slope = (mean(o) - mean(observations[i-1])) / (anchor_points[i] - anchor_points[i-1])
            slopes.append(slope)
    return slopes

def mean(A):
    if len(A) == 0:
        raise Exception("Cannot compute mean for empty set.")
    #return scipy.stats.trim_mean(A, 0.1)
    return np.mean(A)

def getLCApproximation(sizes, scores):
    def ipl(beta):
        a, b, c = tuple(beta.astype(float))
        pl = lambda x: a + b * x **(-c)
        penalty = []
        for i, size in enumerate(sizes):
            penalty.append((pl(size) - scores[i])**2)
        return np.array(penalty)

    a, b, c = tuple(scipy.optimize.least_squares(ipl, np.array([1,1,1]), method="lm").x)
    return lambda x: a + b * x **(-c)


def getStagesAndBudgets(n, k = 10, alpha = .5, gamma = 2, min_anchor_points = 5):
        
    # derive basic sizes
    d = int(np.floor(np.log(.9 * n) / np.log(2)))
    ac = alpha * k
    
    # optimize for c and beta
    c = min_anchor_points + 1
    beta = (alpha / gamma)**(1/(c-min_anchor_points))
    while np.sum([1/(2*beta)**i for i in range(c - min_anchor_points + 1)]) <= 2**(d-c)/alpha:
        c += 1
        beta = (alpha / gamma)**(1/(c-min_anchor_points))
    c -= 1
    beta = (alpha / gamma)**(1/(c-min_anchor_points))
    
    # define anchor points and time budgets
    points = 2**np.array(range(min_anchor_points, c + 1))
    budgets = []
    for i, p in enumerate(points):
        budgets.append(int(np.round(ac / (beta**(c-i - min_anchor_points)))))
    return c, budgets


def LCCV(learner, X, y, target_size=None, r = 0.0, min_stages = 3, timeout=60, enforce_lc_construction=False):
    
    deadline = time.time() + timeout
    print("Running LCCV with timeout",timeout)
    
    n = X.shape[0]
    min_exp = 6
    
    
    if target_size is None:
        target_size = 0.9 * n
        print("Setting target size to " + str(target_size) + " as 90% of the original dataset.")
    
    # compute budgets and phases
    max_exp, repeats = getStagesAndBudgets(X.shape[0], gamma=2)
    print(min_exp, max_exp, repeats)
    anchors = list(range(min_exp, max_exp + 1))
    
    # start LCCV algorithm
    
    observations = []
    mean_observations = []
    
    for stage_id, exp in enumerate(tqdm(anchors)):
        if time.time() > deadline:
            break
        num_examples = 2**exp
        num_evaluations = repeats[stage_id]
        print ("Running stage for " + str(num_examples) + " examples. At most " + str(num_evaluations) + " evaluations will be allowed.")
        
        stabilized = False
        
        observations_at_anchor = []
        variances = []
        
        base_evaluations = 0
        while not stabilized and base_evaluations < num_evaluations and time.time() < deadline:
            observations_at_anchor.append(evaluate(learner, X, y, num_examples))
            
            # criterion 1: low variance in mean estimation
            if len(observations_at_anchor) > 2:
                bootstrap_samples = get_bootstrap_samples(observations_at_anchor, n = 100)
                variances.append(np.var(bootstrap_samples))
                stabilized = variances[-1] < 0.00001
            base_evaluations += 1
        
        observations.append(observations_at_anchor)
        mean_observations.append(mean(observations_at_anchor))
        print("Obtained low-variance result for stage " + str(exp))
        
        # "repair" curve until convex
        print(len(mean_observations))
        expected = np.array(range(len(mean_observations)))
        mismatches = (-np.array(mean_observations)).argsort() != expected
        reevaluations = 0
        while np.count_nonzero(mismatches) and len(observations[stage_id]) < repeats[stage_id] and time.time() < deadline:
            reevaluate = np.where(mismatches)[0]
            print("Found mismatches.", (-np.array(mean_observations)).argsort(), expected, mismatches, "Re-Evaluating", reevaluate)
            any_adjusted = False
            for i in reevaluate:
                if len(observations[i]) < repeats[i]:
                    print("Re-Evaluting", 2**anchors[i])
                    new_score = evaluate(learner, X, y, 2**anchors[i])
                    observations[i].append(new_score)
                    mean_prev = mean_observations[i]
                    mean_observations[i] = mean(observations[i])
                    print("Updated mean", mean_observations[i], "previously was",mean_prev)
                    any_adjusted = True
            if not any_adjusted:
                break
            sorted_anchor_indices = (-np.array(mean_observations)).argsort()
            print(sorted_anchor_indices, mean_observations)
            mismatches = sorted_anchor_indices != expected
            reevaluations += 1
        
        
        slopes = getSlopes([2**e for e in anchors], observations)
        print("slopes:", slopes)
        faulty_segments = [i for i, s in enumerate(slopes) if i > 0 and s < slopes[i-1]]
        while len(faulty_segments) > 0 and base_evaluations < num_evaluations and time.time() < deadline:
            reevaluate = []
            for i in faulty_segments:
                reevaluate = reevaluate + [i-1, i, i+1]
            reevaluate = list(np.unique(reevaluate))
            for i in reevaluate:
                if len(observations[i]) < repeats[i] and time.time() < deadline:
                    print("Re-Evaluting", 2**anchors[i])
                    new_score = evaluate(learner, X, y, 2**anchors[i])
                    observations[i].append(new_score)
                    mean_prev = mean_observations[i]
                    mean_observations[i] = mean(observations[i])
                    print("Updated mean", mean_observations[i], "previously was",mean_prev)
            slopes = getSlopes([2**e for e in anchors], observations)
            print("slopes:", slopes)
            faulty_segments = [i for i, s in enumerate(slopes) if i > 0 and s < slopes[i-1]]
            base_evaluations += 1
        
        #for i, obs in enumerate(slopes):
        
        if len(faulty_segments) == 0:
            print("Obtained stable result for stage " + str(exp))
        else:
            print("Stopped stage " + str(exp) + " with unstable result.")
#        plt.plot(variances)
        
        
        
        # get projections
        if len(slopes) > 0 and stage_id + 1 >= min_stages:
            
            ## convex bound
            last_negative_slope_index = len(slopes) - 1
            while slopes[last_negative_slope_index] > 0:
                last_negative_slope_index -= 1
            slope = slopes[last_negative_slope_index]
            projected_score = mean_observations[stage_id] + (target_size - num_examples) * slope
            print("Projected score for target size " + str(target_size) + " (from anchor " + str(num_examples) + " with mean " + str(mean_observations[stage_id]) + " and slope " + str(slope) + "):", projected_score)
            if projected_score > r:
                print("Impossibly competitive, stopping execution.")
                return np.mean(observations[-1]), anchors, observations
        
            ## inverse power law approximation
            indices = [i for i, exp in enumerate(anchors[:len(mean_observations)]) if exp >= 3][-4:]
            if len(indices) >= 3:
                sizes = np.array([2**e for e in anchors])[indices]
                scores =  np.array(mean_observations)[indices]
                pl_approx = getLCApproximation(sizes, scores)
                #fig, ax = plt.subplots()
                #ax.plot(sizes, scores)
                #domain = np.linspace(0, 10000, 100)
                #ax.plot(domain, pl_approx(domain))
                #plt.show()
    return np.mean(observations[-1]), anchors, observations

In [ ]:
def _DISABLED_fitAndPredict(learner_inst, X_train, y_train, X_test, y_test):
    print("Fitting with " + str(X_train.shape[0]) + " instances.")
    learner_inst.fit(X_train, y_train)
    # compute validation error
    y_hat = learner_inst.predict(X_test)
    mistakes = 0
    print("Testing on " + str(y_hat.shape[0]) + " instances.")
    for i, pred in enumerate(y_hat):
        act = y_test[i]
        if pred != act:
            mistakes += 1            # compute validation error
    return mistakes / X_test.shape[0]
            
            
def cv10(learner, X, y, target_size=None, r = 0.0, min_stages = 3, timeout=60, seed=0):

    train_size = 0.9
    repeats = 10
    deadline = time.time() + timeout
    
    scores = []
    n = X.shape[0]
    num_examples = int(train_size * n)
    
    print("running 10cv for " + str(timeout))
    
    for r in range(repeats):
        print("Repeat", r+1)
        try:
            if deadline <= time.time():
                break
            scores.append(func_timeout(deadline - time.time(), evaluate, (learner, X, y, num_examples, seed+1)))
        except FunctionTimedOut:
            print("TIMED OUT")
            break
            
        except KeyboardInterrupt:
            print("interrupted")
            break
    
    return np.mean(scores) if len(scores) > 0 else np.nan, scores

In [ ]:
def select_model(validation, learners, X, y, timeout_per_evaluation, epsilon):
    validation_func = validation[0]
    validation_result_extractor = validation[1]
    
    hard_cutoff = 2 * timeout_per_evaluation
    r = 1.0
    best_score = 1
    chosen_learner = None
    validation_times = []
    for learner in tqdm(learners):
        print("Checking learner " + str(learner))
        try:
            validation_start = time.time()
            score = validation_result_extractor(validation_func(learner, X, y, r = r, timeout=timeout_per_evaluation))
            validation_times.append(time.time() - validation_start)
            print("Observed score " + str(score) + " for " + str(learner))
            r = min(r, score + epsilon)
            print("r is now:", r)
            if score < best_score:
                best_score = score
                chosen_learner = learner
        except KeyboardInterrupt:
            print("Interrupted, stopping")
            break
        except:
            print("Could NOT evaluate " + str(learner))
    
    return chosen_learner, validation_times

def evaluate_validator(validation, learners, X, y, timeout_per_evaluation, epsilon):
    chosen_learner = select_model(validation, learners, X, y, timeout_per_evaluation, epsilon)
    print("Chosen learner is " + str(chosen_learner) + ". Now computing its definitive performance.")
    true_performance = cv10(chosen_learner, X, y, timeout=60*60*24, seed=0)
    return true_performance

In [ ]:
#openmlid = 23512
openmlid = 31
X, y = getDataset(openmlid)
X.shape

In [ ]:
select_model((LCCV, lambda r: r[0]), [l[0] for l in learners], X, y, 60, 0.0)

In [ ]:
select_model((cv10, lambda r: r[0]), [l[0] for l in learners], X, y, 60, 0.0)

In [ ]:
test_learners = [l[0] for l in learners]
test_learners.reverse()
print(test_learners)
for openmlid in tqdm(datasets_dense):
    print(openmlid)
    X, y = getDataset(openmlid)
    for validator in [cv10, LCCV]:
        print("-------------------------------\n" + validator.__name__ + "\n-------------------------------")
        time_start = time.time()
        score, observations = evaluate_validator((validator, lambda r: r[0]), test_learners, X, y, 60, 0.0)
        runtime = int(np.round(time.time() - time_start))
        print("Runtime was " + str(runtime) + "s. Performance was " + str(score))

In [ ]:
%%time
#evaluate(sklearn.neighbors.KNeighborsClassifier, X, y, int(0.9 * X.shape[0]))
evaluate(sklearn.tree.ExtraTreeClassifier, X, y, int(0.9 * X.shape[0]))

In [ ]:
sizes = []
scores = []
for i, anchor in enumerate(a):
    if i >= len(o):
        break
    sizes.append(2**anchor)
    mean = np.mean(o[i])
    print(len(o[i]), mean)
    scores.append(mean)
print(np.argsort([np.mean(ol) for ol in o]))
fig, ax = plt.subplots()
ax.plot(sizes, scores)
#ax.set_ylim([0,1])
fig, ax = plt.subplots()
ax.boxplot(o)
plt.show()

In [ ]:
import scipy.optimize




indices = np.array(sizes) > 200
pl = getLCApproximation(np.array(sizes)[indices], np.array(scores)[indices])

domain = np.linspace(0, 10000, 100)

fig,ax = plt.subplots()
ax.plot(domain, pl(domain))
ax.plot(sizes, scores)
#ax.set_ylim([0.45, 0.48])

#ipl(np.array([0.5, 0.2, 3]))